In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

from ese.experiment.analysis.analyze_inf import load_cal_inference_stats
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/storage/vbutoi/projects/ionpy/pandas/register.py:47: UserWarning: registration of accessor <class 'ionpy.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/projects/ionpy/pandas/register.py:47: UserWarning: registration of accessor <class 'ionpy.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/projects/ionpy/pandas/register.py:47: UserWarning: registration of accessor <class 'ionpy.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'notNA' for type <class 'pandas.core.series.Series'> is overriding 

In [2]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        - '05_27_24_SW_Pairwise'

options:
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    remove_shared_columns: False
    equal_rows_per_cfg_assert: False 

<IPython.core.display.Javascript object>

In [3]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=False,
)

Error loading submitit directory: /storage/vbutoi/scratch/ESE/inference/05_27_24_SW_Pairwise/SpineWeb_Ensemble_Uncalibrated/submitit
Dropping (datapoint, metric) pairs with NaN metric score. Dropped from 180 -> 180 rows.
Finished loading inference stats.
Log amounts: log.root                                                                                   log_set                                              
/storage/vbutoi/scratch/ESE/inference/05_27_24_SW_Pairwise/SpineWeb_Ensemble_Uncalibrated  20240527_142731-AAOU-5c830be441acb83898d6c552a0ee3207    195
dtype: int64


In [4]:
# import pickle

# # Load all of the preds so we can compare them
# loaded_pred_dict = {}
# for pred_hash_id in inference_df['pred_hash'].unique():
#     # Get the rows corresponding to the hash
#     pred_rows = inference_df[inference_df['pred_hash'] == pred_hash_id]
#     # Get the log set corresponding to the hash
#     log_root = pred_rows['root'].unique()[0]
#     log_set = pred_rows['log_set'].unique()[0]
#     # Load the prediction pickle
#     with open(f'{log_root}/{log_set}/preds/{pred_hash_id}.pkl', 'rb') as f:
#         loaded_pred_dict[pred_hash_id] = pickle.load(f)

In [17]:
import numpy as np
import itertools
unique_subj_ids = inference_df['data_id'].unique()

pw_error_list = []
# Iterate through all the pair-wise comparisons.
for (subj_id_1, subj_id_2) in list(itertools.combinations(unique_subj_ids, 2)):
    if subj_id_1 != subj_id_2:
        # Get the dfs corresponding to these two ids:
        data_id_1_df = inference_df[inference_df['data_id'] == subj_id_1]
        data_id_2_df = inference_df[inference_df['data_id'] == subj_id_2]
        # Get the unique support indices.
        unique_sup_ids_1 = data_id_1_df['sup_idx'].unique()
        unique_sup_ids_2 = data_id_2_df['sup_idx'].unique()
        for (sup_id_1, sup_id_2) in list(itertools.product(unique_sup_ids_1, unique_sup_ids_2)):
            # Get the dfs corresponding to these sup ids
            sup_id_1_df = data_id_1_df[data_id_1_df['sup_idx'] == sup_id_1].reset_index(drop=True)
            sup_id_2_df = data_id_2_df[data_id_2_df['sup_idx'] == sup_id_2].reset_index(drop=True)
            # Now we can get differences in volume by looking at the differences.
            gt_volume_diff = sup_id_2_df['gt_volume'].values[0] - sup_id_1_df['gt_volume'].values[0]
            gt_volume_quot = sup_id_2_df['gt_volume'].values[0] / sup_id_1_df['gt_volume'].values[0]
            # Get the soft pred vol diff
            soft_volume_diff = sup_id_2_df['soft_volume'].values[0] - sup_id_1_df['soft_volume'].values[0]
            soft_volume_quot = sup_id_2_df['soft_volume'].values[0] / sup_id_1_df['soft_volume'].values[0]
            # Get the hard pred vol diff
            hard_volume_diff = sup_id_2_df['hard_volume'].values[0] - sup_id_1_df['hard_volume'].values[0]
            hard_volume_quot = sup_id_2_df['hard_volume'].values[0] / sup_id_1_df['hard_volume'].values[0]
            # Get the Dice information from each of the dataframes
            dice_1 = sup_id_1_df[sup_id_1_df['image_metric'] == 'Dice']['metric_score'].iloc[0]
            dice_2 = sup_id_2_df[sup_id_2_df['image_metric'] == 'Dice']['metric_score'].iloc[0]
            # Get the calibration information from each of the dataframes
            ece_1 = sup_id_1_df[sup_id_1_df['image_metric'] == 'Image_ECE']['metric_score'].iloc[0]
            ece_2 = sup_id_2_df[sup_id_2_df['image_metric'] == 'Image_ECE']['metric_score'].iloc[0]
            # Place all of these into a record
            pair_record = {
                'subj_id_1': subj_id_1,
                'subj_id_2': subj_id_2,
                'sup_id_1': sup_id_1,
                'sup_id_2': sup_id_2,
                'dice_1': dice_1,
                'dice_2': dice_2,
                'ece_1': ece_1,
                'ece_2': ece_2,
                'mean_dice': np.mean([dice_1, dice_2]),
                'mean_ece': np.mean([ece_1, ece_2]),
                'gt_volume_diff': gt_volume_diff,
                'gt_volume_quot': gt_volume_quot,
                'soft_volume_diff': soft_volume_diff,
                'soft_volume_quot': soft_volume_quot,
                'hard_volume_diff': hard_volume_diff,
                'hard_volume_quot': hard_volume_quot,
            }
            # Add the record to the list
            pw_error_list.append(pair_record)

{'subj_id_1': 'Subject04', 'subj_id_2': 'Subject13', 'sup_id_1': 0, 'sup_id_2': 0, 'dice_1': 0.5622857213020325, 'dice_2': 0.6280752420425415, 'ece_1': 0.0026665482073439666, 'ece_2': 0.002120967234077455, 'mean_dice': 0.595180481672287, 'mean_ece': 0.0023937577207107106, 'gt_volume_diff': -47, 'gt_volume_quot': 0.8588588588588588, 'soft_volume_diff': -180.789794921875, 'soft_volume_quot': 0.7379549291607969, 'hard_volume_diff': -137, 'hard_volume_quot': 0.7472324723247232}


ValueError: 